<a href="https://colab.research.google.com/github/Dxmoore/Forest-Cover-Classification-with-Deep-Learning/blob/main/script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np


###Loading Data###

#import data
data = pd.read_csv('cover_data.csv')

#view data columns and types
data.info()

# Check for NaN values in the 'class' column-- found one row with nan value 
nan_values = data['class'].isna().sum()

# Print the number of NaN values
print("Number of NaN values in 'class' column:", nan_values)

# Drop rows with NaN values in the 'class' column
data = data.dropna(subset=['class'])

#view distribution of the target column 'death event'
print('Classes and number of values in the dataset', Counter(data['class'])) 

#create variable for label and features column
y = data['class']
x = data.drop(['class'], axis = 1)




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251506 entries, 0 to 251505
Data columns (total 55 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Elevation                           251506 non-null  int64  
 1   Aspect                              251506 non-null  int64  
 2   Slope                               251506 non-null  int64  
 3   Horizontal_Distance_To_Hydrology    251506 non-null  int64  
 4   Vertical_Distance_To_Hydrology      251506 non-null  int64  
 5   Horizontal_Distance_To_Roadways     251506 non-null  int64  
 6   Hillshade_9am                       251506 non-null  int64  
 7   Hillshade_Noon                      251506 non-null  int64  
 8   Hillshade_3pm                       251506 non-null  int64  
 9   Horizontal_Distance_To_Fire_Points  251506 non-null  int64  
 10  Wilderness_Area1                    251506 non-null  int64  
 11  Wilderness_Area2          

The dataset contains a great imbalance in classifications, leaning heavily towards classes 1 and 2 (Spruce/Fir
Lodgepole Pine). 

In [28]:
###Data Preprocessing###


#split the data
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.2, random_state = 40)


#scale the numeric training and test feature values
numerical_columns = data.select_dtypes(include=['int64']).columns
ct = ColumnTransformer([("only numeric", StandardScaler(), x.columns)], remainder='passthrough')#used x.columns since all data is in int64
X_train_scaled = ct.fit_transform(X_train)
X_test_scaled = ct.transform(X_test)







In [29]:
###Create Model###


# Subtract 1 from class labels -> class lables must be zero indexed to use with sparse categorical crossentrophy
Y_train -= 1


# Define the early stopping callback
early_stopping = EarlyStopping(monitor='loss', patience=10, verbose=1)

#create model instance with input, hidden and output layers 
model = Sequential()
model.add(InputLayer(input_shape=(X_train_scaled.shape[1],))) #input
model.add(Dense(12, activation='relu')) #hidden
model.add(Dense(7, activation='softmax')) #output
# Use sparse categorical cross-entropy as the loss function
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #compile model
model.fit(X_train_scaled, Y_train, epochs=100, batch_size=64, callbacks = [early_stopping]) # fit model






Epoch 1/100
3144/3144 [==============================] - 9s 3ms/step - loss: 0.6521 - accuracy: 0.7413
Epoch 2/100
3144/3144 [==============================] - 9s 3ms/step - loss: 0.5295 - accuracy: 0.7738
Epoch 3/100
3144/3144 [==============================] - 9s 3ms/step - loss: 0.5042 - accuracy: 0.7824
Epoch 4/100
3144/3144 [==============================] - 8s 3ms/step - loss: 0.4903 - accuracy: 0.7878
Epoch 5/100
3144/3144 [==============================] - 9s 3ms/step - loss: 0.4822 - accuracy: 0.7927
Epoch 6/100
3144/3144 [==============================] - 9s 3ms/step - loss: 0.4755 - accuracy: 0.7955
Epoch 7/100
3144/3144 [==============================] - 8s 3ms/step - loss: 0.4704 - accuracy: 0.7976
Epoch 8/100
3144/3144 [==============================] - 8s 3ms/step - loss: 0.4670 - accuracy: 0.7991
Epoch 9/100
3144/3144 [==============================] - 9s 3ms/step - loss: 0.4649 - accuracy: 0.7998
Epoch 10/100
3144/3144 [==============================] - 9s 3ms/step - l

In [32]:
### Generate a classification report ###
Y_test -= 1
loss, acc = model.evaluate(X_test_scaled, Y_test)
y_estimate = model.predict(X_test_scaled, verbose=0)
y_estimate = np.argmax(y_estimate, axis=1)
print(classification_report(Y_test, y_estimate))

1572/1572 [==============================] - 4s 3ms/step - loss: 0.4319 - accuracy: 0.8173
              precision    recall  f1-score   support

         0.0       0.76      0.70      0.73     14477
         1.0       0.85      0.90      0.87     31276
         2.0       0.68      0.83      0.75      1252
         3.0       0.82      0.74      0.78       433
         4.0       0.76      0.50      0.60      1130
         5.0       0.60      0.49      0.54       978
         6.0       0.86      0.66      0.74       755

    accuracy                           0.82     50301
   macro avg       0.76      0.69      0.72     50301
weighted avg       0.81      0.82      0.81     50301

